In [12]:
import torch
import torchvision.datasets as datasets # 이미지 관련 파이토치 라이브러리
import torchvision.transforms as transforms # 전처리 기능 제공하는 라이브러리
import torch.nn as nn # torch 내의 세부적인 기능 불러오기 (신경망 기술, 손실함수 등)
import torch.optim as optim # 최적화 문제
import random

In [ ]:
# CPU/GPU
# GPU 쓰려구. CNN은 GPU에서 효율적인 연산 가능
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available.')

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 5
batch_size = 100

In [ ]:
# MNIST dataset
mnist_train = datasets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = datasets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
# 일반적으로 데이터셋은 이미지와 라벨이 동시에 들어있는 튜플(tuple) 형태 (이미지, 라벨)
# mnist_train[0]은 학습 데이터의 첫 번째 데이터로 이미지 한 장과 라벨 숫자 하나가 저장되어 있음
# 즉, mnist_train[0][0]은 이미지이며 mnist_train[0][1]은 라벨임
print(mnist_train.classes)

In [ ]:
print(mnist_train[10])

In [ ]:
# DataLoader는 데이터를 미니 배치 형태로 만들어 줌
# 따라서 배치 사이즈 및 셔플 여부 등을 선택할 수 있음
# 여기서 배치 사이즈는 한번 학습을 할 때마다 이미지가 100개씩 들어감을 의미
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
data_iter = iter(train_loader)
images, labels = data_iter.next()

print(images.size())
print(labels) # 배치크기 X 채널 수 X 너비 X 높이 

In [7]:
# MLP Model
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        
        self.layer1 = nn.Linear(784, 256, bias = True)
        self.layer2 = nn.Linear(256, 256, bias = True)
        self.layer3 = nn.Linear(256, 10, bias = True)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.relu(out)

        return out

In [8]:
# MLP 모델 선언
# GPU 연산을 할 수 있도록 반드시 to(device)를 써줘야함
model = MLP().to(device)

In [ ]:
print(model)

In [9]:
# loss 함수, 옵티마이저 정의
criterion = nn.CrossEntropyLoss().to(device) # 다중 분류 문제라서 사용. softmax 계산까지 포함됨
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # 업데이트 되어야하는 파라미터 첫번째 인자로 넘겨주기

In [ ]:
total_batch = len(train_loader)
for epoch in range(training_epochs):
    avg_loss = 0

    for X, Y in train_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, Y)
        loss.backward()
        optimizer.step()

        avg_loss += loss / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.9f}'.format(avg_loss))

print('Learning finished')

In [ ]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())